In [1]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [2]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [3]:
import pickle
import pandas as pd
import numpy as np
from utils import *

In [4]:
test = pd.read_csv('../input/test.csv')
test = transformDf(test)

In [5]:
with open('../input/parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

with open('../input/preprocessed_data.pkl', "rb") as f:
    pickle.dump(X, f, protocol=2)
    pickle.dump(y_a, f, protocol=2)
    pickle.dump(y_t, f, protocol=2)
    pickle.dump(X_train, f, protocol=2)
    pickle.dump(y_a_train, f, protocol=2)
    pickle.dump(y_t_train, f, protocol=2)
    pickle.dump(X_test, f, protocol=2)
    pickle.dump(y_a_test, f, protocol=2)
    pickle.dump(y_t_test, f, protocol=2)

# Utils

In [7]:
def inverseActivity(predictions, target_indices_char):
    maxPrediction = 0
    symbol = ''
    i = 0;
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

In [8]:
def inverseTime(predictions):
    return predictions*divisor

# Predict

In [9]:
model = load_model('output_files/models/model_31-0.95.h5')

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


In [10]:
test_groupByCase = test.groupby(['CaseID'])

In [12]:
pred_a = []
pred_t = []

gt_a = []
gt_b = []
for case, group in test_groupByCase:
    if group.shape[0] == 3:     
        case_groupby = group.groupby(['CaseID'])
        #input
        X = vectorizeInput(case_groupby, maxlen, num_features, chartoindice, divisor, divisor2)
        
        #output
        y_a = one_hot_encode(case_groupby, targetchartoindice)
        y_t = nomalize(case_groupby, divisor)
        
        #prediction
        pred = model.predict(X, verbose=0)
        pred_a = pred[0]
        pred_t = pred[1]
    break

In [13]:
pred_a

array([[  1.36205763e-01,   8.33198071e-01,   2.33788341e-02,
          7.38655501e-07,   5.74752269e-03,   1.29288310e-04,
          6.46961125e-05,   3.25474175e-05,   8.43285125e-06,
          1.23411370e-03],
       [  6.70459936e-04,   2.74350177e-02,   7.42312074e-01,
          9.51916022e-08,   2.20601827e-01,   4.38199332e-03,
          8.42419744e-04,   1.59177725e-05,   1.15764688e-03,
          2.58256309e-03],
       [  3.34504148e-04,   1.67101733e-02,   4.77547944e-02,
          1.77308497e-08,   2.70581176e-03,   1.21021891e-04,
          2.65391554e-05,   1.09832190e-05,   1.16450501e-05,
          9.32324529e-01]], dtype=float32)

In [14]:
y_a

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32)

In [15]:
y_t

array([ 1.3322443 ,  1.21594656,  0.        ])

In [19]:
inverseActivity(pred_a, targetchartoindice)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [20]:
inverseTime(pred_t)

array([[-81231.125    ],
       [-74033.71875  ],
       [-88864.4921875]], dtype=float32)

In [17]:
pred_t

array([[-0.38513583],
       [-0.35101125],
       [-0.42132741]], dtype=float32)

# Transform

# Evaluate